# Regression without A (SEER)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import metrics
import statsmodels.api as sm

In [194]:
pd.set_option('display.max_rows', 500)

In [3]:
df = pd.read_csv('SEER_expanded_v1210.csv')

data_M0 = df[(df['Metastasis']=='M0') & (df['Stage']!='0')]
data_M0 = data_M0.drop(columns=['Metastasis', 'Death_All_Cause_ext']).reset_index()

In [4]:
X_M0 = pd.get_dummies(data_M0[['DURATION', 'Sex', 'Yr_dx', 'Laterality', 'reg_nod_pos', 'reg_nod_ex',
       'Ulceration', 'Mitotic rate', 'Site_rec_WHO08',
       'Origin', 'AYA_site_rec20', 'Tot_num_in_situ', 'Tot_num_benin',
       'Mar_stat', 'Age_dx', 'Extent', 'Surg_LN', 'Surg_primsite',
       'Surg_oth', 'Tumor', 'Positive_Node', 'Stage', 'YEAR', 'AGE']], drop_first=False)
w_M0 = data_M0['EXPO']
Y_M0 = data_M0['Death_skin']

In [5]:
x_M0_train, x_M0_test, y_M0_train, y_M0_test, w_M0_train, w_M0_test = train_test_split(X_M0, Y_M0, w_M0, test_size=0.2, stratify=Y_M0)

In [6]:
x_M0_train = sm.add_constant(x_M0_train)
x_M0_test = sm.add_constant(x_M0_test)

In [7]:
# Selected variables, without protected
col = ['const', 'DURATION', 'AGE', 'reg_nod_pos',
       'Mitotic rate', 'Tot_num_in_situ', 'Laterality_Bilateral',
       'Laterality_Not a paired site',
       'Laterality_One side', 'Laterality_Paired site',
       'Laterality_Paired site midline tumor', 'Laterality_Right origin',
       'Ulceration_Missing',
       'Ulceration_Ulceration', 'Site_rec_WHO08_Male Genital Organs',
       'Site_rec_WHO08_Vulva',
       'Extent_Distant',
       'Extent_Missing', 'Extent_Regional', 'Surg_primsite_Missing',
       'Surg_primsite_No surgery', 'Tumor_Missing', 'Tumor_T0',
       'Tumor_T2', 'Tumor_T3', 'Tumor_T4', 'Positive_Node_Missing',
       'Positive_Node_N1', 'Positive_Node_N2',
       'Positive_Node_N3']

In [8]:
mod_all_M0 = sm.GLM(y_M0_train, x_M0_train[col], family=sm.genmod.families.family.Binomial(), freq_weights=w_M0_train)

In [9]:
res_all_M0 = mod_all_M0.fit()

print(res_all_M0.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:             Death_skin   No. Observations:               430764
Model:                            GLM   Df Residuals:                404743.42
Model Family:                Binomial   Df Model:                           29
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -14330.
Date:                Fri, 04 Nov 2022   Deviance:                       28660.
Time:                        14:55:33   Pearson chi2:                 3.51e+05
No. Iterations:                     9   Pseudo R-squ. (CS):            0.01612
Covariance Type:            nonrobust                                         
                                           coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------


In [10]:
data_pred = data_M0.loc[x_M0_test.index, :]

In [11]:
data_pred['Y_pred proba'] = res_all_M0.predict(x_M0_test[col])

### Coefficients of the model

In [16]:
plt.figure(figsize=(7,10))
plt.barh(res_all_M0.params.index.tolist()[1:], res_all_M0.params[1:], xerr=res_all_M0.bse[1:])
# plt.savefig('seer_coeff_withoutA_v411.png')

### Performance metrics

In [13]:
fpr, tpr, thresholds = metrics.roc_curve(data_pred['Death_skin'], data_pred['Y_pred proba'], sample_weight=w_M0_test)

In [17]:
plt.plot(fpr, tpr, label='AUC: ' + str(np.round(metrics.roc_auc_score(data_pred['Death_skin'], data_pred['Y_pred proba'], sample_weight=w_M0_test), 4)))
plt.plot([0, 1], [0, 1], linestyle='dashed')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.legend()
# plt.savefig('ROC_withoutA_v411.png')

### Threshold for classification

In [18]:
# Proportion of deaths in the dataset
data_M0['Death_skin'].value_counts()[1]/data_M0.shape[0]

0.007025643692335121

In [19]:
for i in np.linspace(0.104, 0.106, 4):
    data_pred['Y_pred' + str(i)] = data_pred['Y_pred proba'] > i
    print(i, data_pred['Y_pred' + str(i)].value_counts()[1]/data_pred.shape[0])

In [ ]:
# Threshold of 0.10533333333333333 gives the same proportion for predicted deaths

### Fairness metrics

In [203]:
Y = 'Death_skin'
Y_pred = 'Y_pred0.10533333333333333'
A = 'Mar_stat'
list_sens = data_pred[A].unique()

Global fairness metrics

In [204]:
FPs=metrics.confusion_matrix(data_pred[Y], data_pred[Y_pred])[1, 0]
FNs=metrics.confusion_matrix(data_pred[Y], data_pred[Y_pred])[0, 1]
TPs=metrics.confusion_matrix(data_pred[Y], data_pred[Y_pred])[0, 0]
TNs=metrics.confusion_matrix(data_pred[Y], data_pred[Y_pred])[1, 1]

In [94]:
ARs = (TPs+FPs)/(TPs+FPs+FNs+TNs)
TPRs = TPs/(TPs+FNs)
FPRs = FPs/(FPs+TNs)
print(ARs, TPRs, FPRs)

Group-wise fairness metrics

In [207]:
FPs_A=[]
FNs_A=[]
TPs_A=[]
TNs_A=[]

for sens in list_sens:
    if metrics.confusion_matrix(data_pred[data_pred[A] == sens][Y], data_pred[data_pred[A] == sens][Y_pred]).shape == (2, 2):
        FPs_A.append(metrics.confusion_matrix(data_pred[data_pred[A] == sens][Y], data_pred[data_pred[A] == sens][Y_pred])[1, 0])
        FNs_A.append(metrics.confusion_matrix(data_pred[data_pred[A] == sens][Y], data_pred[data_pred[A] == sens][Y_pred])[0, 1])
        TPs_A.append(metrics.confusion_matrix(data_pred[data_pred[A] == sens][Y], data_pred[data_pred[A] == sens][Y_pred])[0, 0])
        TNs_A.append(metrics.confusion_matrix(data_pred[data_pred[A] == sens][Y], data_pred[data_pred[A] == sens][Y_pred])[1, 1])
    else:
        print(sens)

In [217]:
# for Origin, there are only TP
data_pred[data_pred['Origin'] == 'Missing'][Y_pred].value_counts(), data_pred[data_pred['Origin'] == 'Missing'][Y].value_counts()

(False    2344
 Name: Y_pred0.10533333333333333, dtype: int64,
 0    2344
 Name: Death_skin, dtype: int64)

In [208]:
ARs_A = np.array([(TPs_A[i]+FPs_A[i])/(TPs_A[i]+FPs_A[i]+FNs_A[i]+TNs_A[i]) for i in range(len(FPs_A))])
TPRs_A = np.array([TPs_A[i]/(TPs_A[i]+FNs_A[i]) for i in range(len(FPs_A))])
FPRs_A = np.array([FPs_A[i]/(FPs_A[i]+TNs_A[i]) for i in range(len(FPs_A))])
print(ARs_A, TPRs_A, FPRs_A)
print(np.var(ARs_A), np.var(TPRs_A), np.var(FPRs_A))

In [171]:
# # Origin
# list_sens, list_sens[[0,1,3,4,5,2]]

(array(['White', 'Hispanic', 'Missing', 'Black',
        'Asian or Pacific Islander', 'American Indian/AK Native'],
       dtype=object),
 array(['White', 'Hispanic', 'Black', 'Asian or Pacific Islander',
        'American Indian/AK Native', 'Missing'], dtype=object))

In [186]:
list_sens#, list_sens[[0,2,1,4,3,5,6]]

array(['Female', 'Male'], dtype=object)

In [214]:
# plt.figure(figsize=(2,3))
plt.bar(list_sens, FPRs_A)
plt.ylim((0.7, 1.01))
plt.xticks(rotation=45, ha="right")
# plt.savefig('seer_FPR_marstat_withoutA_v1011.png')